<a href="https://colab.research.google.com/github/fkaimphilat/YAXHSE_coursera_ml/blob/master/hw10/hw10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Домашнее задание № 10. Генерация текста**

# Задание 1 (8 баллов).

Попробуйте дообучать GPT на каком-то другом тексте (можете попробовать любые стихи или какие-то специфичные вещи вроде анекдотов, теорий заговоров, постов в помоечных телеграм каналах, текстов журналистов и СМИ с выразительным стилем). 
Попробуйте разные методы и параметры генерации (beam search, температура, top_k и тп). Сохраните в тетрадке несколько хороших сгенерированных текстов


In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 30.2 MB/s 
     |████████████████████████████████| 596 kB 68.7 MB/s 
     |████████████████████████████████| 86 kB 7.2 MB/s 
     |████████████████████████████████| 6.6 MB 64.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import Trainer, TrainingArguments
from transformers import TextDataset, DataCollatorForLanguageModeling
import torch

In [6]:

DEVICE = torch.device("cuda:0")

model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, use_cache=False).to(DEVICE)

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

### **Дообучаем GPT на небольшом сборнике цитат Камы Пули - его характерная речь и язык достаточно необычны и полны особенностей, что должно отразиться в сгенерированных текстах.**

In [10]:
train_path = '/content/kama_quotes.txt'

In [11]:
# Создание датасета
train_dataset = TextDataset( tokenizer=tokenizer,file_path=train_path,block_size=64, 
                            overwrite_cache=True)
  
# специальный класс который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [13]:
training_args = TrainingArguments( 
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,  
    gradient_accumulation_steps=16, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [14]:
trainer.train()

***** Running training *****
  Num examples = 11
  Num Epochs = 100
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 16
  Total optimization steps = 100


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=100, training_loss=0.0930668830871582, metrics={'train_runtime': 25.5773, 'train_samples_per_second': 43.007, 'train_steps_per_second': 3.91, 'total_flos': 35927654400000.0, 'train_loss': 0.0930668830871582, 'epoch': 100.0})

In [49]:
text1 = 'Я скажу '
input_ids1 = tokenizer.encode(text1, return_tensors="pt").to(DEVICE)

text2 = 'Слышал '
input_ids2 = tokenizer.encode(text2, return_tensors="pt").to(DEVICE)

## **ArgMax**

In [50]:
with torch.no_grad():
    out1 = model.generate(input_ids1, do_sample=False, max_length=50)
    out2 = model.generate(input_ids2, do_sample=False, max_length=50)
    
generated_text1 = list(map(tokenizer.decode, out1))[0]
generated_text2 = list(map(tokenizer.decode, out2))[0]
print()
print(generated_text1, '\n', generated_text2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Я скажу  - не надо драматизировать.
Я скажу - не надо драматизировать. Вышел ты один. Враг - это отражение. Уходишь бля, бля, бля, бля.
Я скажу - не надо драматизировать 
 Слышал  -  у  тебя  есть  девушка  с  деньгами.  Она  тебе  поможет.
Она тебе не поможет. Она тебе не поможет с проституткой.
Слышал, у тебя есть крошка -


### **Хотя уже немного похоже на язык Камы Пули, но до реальной речи не дотягивает. Здесь можно заметить повторение, что, как я предполагаю, есть следствие неполноты данных - наш "корпус" все-таки совсем крошечный.**

## **BeamSearch**

### *Высокий num_beams*

In [53]:
out1 = model.generate(input_ids1, do_sample=False, num_beams=20, max_length=60)
out2 = model.generate(input_ids2, do_sample=False, num_beams=20, max_length=60)

generated_text1 = list(map(tokenizer.decode, out1))[0]
generated_text2 = list(map(tokenizer.decode, out2))[0]
print()
print(generated_text1, '\n', generated_text2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Я скажу  - жи есть.
Есть дерьмо, которое отталкивает.
Есть дерьмо, которое отталкивает.
Есть дерьмо, которое отталкивает.
Есть дерьмо, которое отталкивает.
Есть дерьмо, которое отталкивает.
Есть дерьмо, которое отталкивает.
Есть дерьмо, которое 
 Слышал 
Слышал 
Слышал 
Слышал 
Слышал 
Слышал 
Слышал 
Слышал 
Слышал 
Слышал 
Слышал 
Слышал 



### **Высокий num_beams ломает нормальную генерацию, хотя в начале есть что-то осмысленное.**

### *Низкий num_beams*

In [54]:
out1 = model.generate(input_ids1, do_sample=False, num_beams=5, max_length=60)
out2 = model.generate(input_ids2, do_sample=False, num_beams=5, max_length=60)

generated_text1 = list(map(tokenizer.decode, out1))[0]
generated_text2 = list(map(tokenizer.decode, out2))[0]
print()
print(generated_text1, '\n', generated_text2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Я скажу  - не надо драматизировать. Вышел ты один. Враг - это отражение. Выходи один.
Не надо драматизировать. Вышел ты один. Враг - это отражение. Выходи один.
Не надо драматизировать. Вышел ты один. Вра 
 Слышал  анекдот про двух зайцев: во-первых, не надо драматизировать. Во-вторых, не надо драматизировать. Вышел ты один. Враг - это отражение. Враг - это отражение. Враг - это отражение. Враг - это отражение. Вра


### **Низкий num_beams уже делает язык более интресным, но как-то все равно слишком много "драмы".**

## **Сэмплирование с Температурой**

### *Низкая температура*

In [55]:
out1 = model.generate(input_ids1, do_sample=True, temperature=0.01, max_length=50)
out2 = model.generate(input_ids2, do_sample=True, temperature=0.01, max_length=50)

generated_text1 = list(map(tokenizer.decode, out1))[0]
generated_text2 = list(map(tokenizer.decode, out2))[0]
print()
print(generated_text1, '\n', generated_text2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Я скажу  - не надо драматизировать.
Я скажу - не надо драматизировать. Вышел ты один. Враг - это отражение. Уходишь бля, бля, бля, бля.
Я скажу - не надо драматизировать 
 Слышал  -  у  тебя  есть  девушка  с  деньгами.  Она  тебе  поможет.
Она тебе не поможет. Она тебе не поможет с проституткой.
Слышал, у тебя есть крошка -


### *Температура побольше*

In [56]:
out1 = model.generate(input_ids1, do_sample=True, temperature=0.6, max_length=50)
out2 = model.generate(input_ids2, do_sample=True, temperature=0.6, max_length=50)

generated_text1 = list(map(tokenizer.decode, out1))[0]
generated_text2 = list(map(tokenizer.decode, out2))[0]
print()
print(generated_text1, '\n', generated_text2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Я скажу  - это ты у меня в долгу.
Не надо драматизировать. Вышел ты один. Враг - это отражение. Уходишь бля, разносишь жи есть.
Есть отражение в мелочах. Есть отражение в мело 
 Слышал  ты в  Китае живешь?
Я в России родился.
Азия - это просто игрушки.
Ничего личного - просто бизнес.
Слышал, в Китае есть улица, по которой можно ходить?
По ней и


### *Вообще большая температура жесть...*

In [57]:
out1 = model.generate(input_ids1, do_sample=True, temperature=1.5, max_length=50)
out2 = model.generate(input_ids2, do_sample=True, temperature=1.5, max_length=50)

generated_text1 = list(map(tokenizer.decode, out1))[0]
generated_text2 = list(map(tokenizer.decode, out2))[0]
print()
print(generated_text1, '\n', generated_text2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Я скажу  - ты даже не поймешь, здесь я или там, и в конце-концов - уже даже не знаю, с какой стороны к тебе подкрасться - со спины, с тыла, или снизу - сбоку».
Да хрен с тобой 
 Слышал  тебя приятель  в магазине электроники.
Ты с нами или с крошками в тундре. Пока не определились, кто ты есть, не обессудь.
Америкосы в каменном веке отдыхают.



### **Как ни странно, высокая температура дала очень не плохой сгенерированный текст, однако в тексте заметны ошибки.**

## **top_k**


In [58]:
out1 = model.generate(input_ids1, 
                     do_sample=True,
                     temperature=1.5,
                     top_k=20,
                     max_length=50,
                    )

out2 = model.generate(input_ids2, 
                     do_sample=True,
                     temperature=1.5,
                     top_k=20,
                     max_length=50,
                    )

generated_text1 = list(map(tokenizer.decode, out1))[0]
generated_text2 = list(map(tokenizer.decode, out2))[0]
print()
print(generated_text1, '\n', generated_text2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Я скажу  тебе вещи, которых я в тебя не смыслю. Не хочу обидеть тех, кого знаю, - они друзья - или враги, или любовники. Все мы разные и не всегда люди равные.
Любой человек создан для жизни, 
 Слышал  я одного человека с большой буквы, кто бы знал, с какой легкостью я к этому не прикоснусь!
Слышал я, что один Бог создал одного Люцифа с Большой Грешной Книгой Судеб.
Не обессудь


### **Изменение параметра top_k уже позволило создать текст с меньшим количеством ошибок - он не только похож на реальные цитаты Камы Пули, но и имеет какой-то смысл. В конце концов, все мы разные и не всегда люди равные.**

# Задание  2 (2 балла)


**1) В каких статья были представлены GPT-1, GPT-2, GPT-3?**

GPT-1: https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf

GPT-2: https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf

GPT-3: https://arxiv.org/pdf/2005.14165.pdf


**2) Как собирался обучающий корпус для GPT-3? Каким образом создатели старались обеспечить высокое качество текстов в обучающей выборке?**

Шестьдесят процентов набора данных для предварительного обучения GPT-3 составляет отфильтрованная версия Common Crawl, состоящая из 410 миллиардов закодированных лексем в байтовых парах. Другими источниками являются 19 миллиардов лексем из WebText2, что составляет 22% от общего количества, 12 миллиардов лексем из Books1, что составляет 8%, 55 миллиардов лексем из Books2, что составляет 8%, и 3 миллиарда лексем из Wikipedia, что составляет 3%. GPT-3 был обучен на сотнях миллиардов слов и способен выполнять кодирование на CSS, JSX, Python и т.д. В обзоре 2022 года вновь подчеркивается, что обучение продолжает включать обзор Википедии.

Было предпринято 3 шага для улучшения среднего качества датасетов: была загружена версия CommonCrawl и отфильтрована на основе сходства с рядом высококачественных опорных корпусов, была выполнена fuzzy дедупликация на уровне документов, внутри и между наборами данных, чтобы предотвратить избыточность и сохранить целостность удерживаемого валидационного набора в качестве точной меры overfitting, а также добавлены известные качественные корпуса в обучающую выборку (выше), чтобы дополнить CommonCrawl и увеличить его разнообразие.
